Source: https://github.com/philschmid/huggingface-llama-2-samples/blob/master/training/sagemaker-notebook.ipynb

# Deploy LLaMA 2 on Amazon SageMaker

In this script we will deploy an already finetuned Llama model.

## 1. Setup Development Environment

In [56]:
!pip install --upgrade pip

In [57]:
!pip install "transformers==4.30.2" "datasets[s3]==2.13.0" sagemaker --upgrade --quiet

If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.



In [58]:
import sagemaker
import boto3
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
SESS = sagemaker.Session()
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and SESS is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = SESS.default_bucket()

try:
    ROLE = sagemaker.get_execution_role()
except ValueError:
    IAM = boto3.client('iam')
    ROLE = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

SESS = sagemaker.Session(default_bucket=sagemaker_session_bucket)
S3_CLIENT = boto3.client('s3')
REGION = SESS.boto_region_name

print(f"sagemaker role arn: {ROLE}")
print(f"sagemaker bucket: {SESS.default_bucket()}")
print(f"sagemaker session region: {REGION}")


sagemaker role arn: arn:aws:iam::275461957965:role/service-role/AmazonSageMaker-ExecutionRole-20230627T145146
sagemaker bucket: sagemaker-us-east-1-275461957965
sagemaker session region: us-east-1


After we processed the datasets we are going to use the new [FileSystem integration](https://huggingface.co/docs/datasets/filesystems) to upload our dataset to S3. We are using the `sess.default_bucket()`, adjust this if you want to store the dataset in a different S3 bucket. We will use the S3 path later in our training script.

## 2. Get the model artifacts URI from S3 for the training job

In order to deploy a finetune model we need to find its weights in SageMaker. The all training jobs live [here](https://us-east-1.console.aws.amazon.com/sagemaker/home?region=us-east-1#/jobs). In order for the below fudntion to work you need to define two arguments: 
- `base_job_name` (the name of the job),  
- `model_id` (after you choose the right training job, find Job Settings -> Hyperparameters -> model_id. Really you navigate to the job page and then crlt+f 'model_id'). 

In [59]:
import boto3
import sagemaker

base_job_name = 'huggingface-qlora-2024-01-19-01-46-55-2024-01-19-01-46-58-278'
# base_job_name = 'huggingface-qlora-2023-10-24-17-28-34-2023-10-24-17-30-01-128'
model_id = 'meta-llama/Llama-2-13b-chat-hf'


def get_latest_training_job_artifact(base_job_name):
    sagemaker_client = boto3.client('sagemaker', region_name=REGION)
    response = sagemaker_client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', SortOrder='Descending')
    training_job_arn = response['TrainingJobSummaries'][0]['TrainingJobArn']
    print('Training_job_arn: ' + training_job_arn)
    training_job_description = sagemaker_client.describe_training_job(TrainingJobName=training_job_arn.split('/')[-1])
    return training_job_description['ModelArtifacts']['S3ModelArtifacts']

try:
    model_data_uri = huggingface_estimator.model_data
except:
    # Retrieve artifact url when kernel is restarted
    model_data_uri = get_latest_training_job_artifact(base_job_name)
model_data_uri

# !aws s3 cp {model_data_uri} llama.tar.gz

Training_job_arn: arn:aws:sagemaker:us-east-1:275461957965:training-job/huggingface-qlora-2024-01-19-01-46-55-2024-01-19-01-46-58-278


's3://sagemaker-us-east-1-275461957965/huggingface-qlora-2024-01-19-01-46-55-2024-01-19-01-46-58-278/output/model.tar.gz'

## 3. Deploy Model

You can deploy your fine-tuned LLaMA model to a SageMaker endpoint and use it for inference. Check out the [Deploy Falcon 7B & 40B on Amazon SageMaker](https://www.philschmid.de/sagemaker-falcon-llm) and [Securely deploy LLMs inside VPCs with Hugging Face and Amazon SageMaker](https://www.philschmid.de/sagemaker-llm-vpc) for more details.

### Retrieve the new Hugging Face LLM DLC

In [60]:
from sagemaker.huggingface import get_huggingface_llm_image_uri

# retrieve the llm image uri
llm_image = get_huggingface_llm_image_uri(
  "huggingface",
  version="0.9.3"
)

# print ecr image uri
print(f"llm image uri: {llm_image}")

llm image uri: 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.0.1-tgi0.9.3-gpu-py39-cu118-ubuntu20.04


### Deploy the modelt to SageMaker

In [61]:
import json
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.huggingface import get_huggingface_llm_image_uri
from huggingface_hub import HfFolder

# sagemaker config
instance_type = "ml.g5.12xlarge"
number_of_gpu = 4 # this has to be 1 for falcon-7b models since they don't support model sharding
health_check_timeout = 100

# Define Model and Endpoint configuration parameter
config = {
  'HF_MODEL_ID': "/opt/ml/model", # huggingface model id
  'SM_NUM_GPUS': json.dumps(number_of_gpu), # Number of GPU used per replica
  'MAX_INPUT_LENGTH': json.dumps(1024), # Max length of input text
  'MAX_TOTAL_TOKENS': json.dumps(2048), # Max length of the generation (including input text)
  'NUM_SHARD': json.dumps(1), # comment this out when not using Falcon-7b
  'HF_MODEL_QUANTIZE': "bitsandbytes",# Comment in to quantize, 
  # 'HUGGING_FACE_HUB_TOKEN': HfFolder.get_token()
}

# create HuggingFaceModel with the image uri
llm_model = HuggingFaceModel(
  model_data=model_data_uri,  #finetuned- model
  role=ROLE,
  image_uri=llm_image,
  env=config
)


# deploy model to SageMaker Inference
predictor = llm_model.deploy(
   initial_instance_count=1,
   instance_type=instance_type,
   container_startup_health_check_timeout=health_check_timeout,
)


endpoint_name = predictor.endpoint_name
print(f"Endpoint Name: {endpoint_name}")

----------!Endpoint Name: huggingface-pytorch-tgi-inference-2024-01-19-19-18-47-920


In [62]:
print(predictor)

HuggingFacePredictor: {'endpoint_name': 'huggingface-pytorch-tgi-inference-2024-01-19-19-18-47-920', 'sagemaker_session': <sagemaker.session.Session object at 0x7fd53b8807c0>, 'serializer': <sagemaker.base_serializers.JSONSerializer object at 0x7fd53e377f10>, 'deserializer': <sagemaker.base_deserializers.JSONDeserializer object at 0x7fd53e377e50>}


### Check if the end point is working

In [63]:
import scripts.prompt_utils
from scripts.prompt_utils import format_prompt_instruct

# define question and add to messages
system_prompt = scripts.prompt_utils.system_prompt_descriptor
user_instruction = "title: Devious Women's Femme 3020 34 inches Boots, brand: Devious, price: $68.76 - $145.50, feature: ['Synthetic', 'Shaft measures approximately 26 inches from arch', 'Heel Height: 8inches ', 'Shaft Height: 22 inches (size 7)', 'Fit: True to Size', 'Features of this item include: Sexy', 'Shipping Weight:\n                    \n6.5 pounds (View shipping rates and policies)'] [/INST] ['Shaft Height: Approx. 26\'\' Tall. Heel Height: Approx. 8\'\' Tall. The ultimate extension for those who live the lifestyle or those who just wish to spice it up again, Devious offers a fierce line of footwear to play with. The Femme-3020 boot has a unique block heel and 1 1/4 inches platform with a lace up thigh high design."
# user_instruction = "What is the difference between contribution margin costing and traditional costing?"

prompt = format_prompt_instruct(
    user_instruction = user_instruction,
    system_prompt = system_prompt,
)

body = {
    "inputs":  prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.6,
    "temperature": 0.4,
    "top_k": 4,
    "max_new_tokens": 1000,
    "repetition_penalty": 1.03,
    "stop": ["</s>"]
  }
}


chat = predictor.predict(body)
print(chat[0]["generated_text"])

Indulge in the ultimate experience of power and seduction with the Devious Women's Femme-3020 34 inches Boots. These boots are designed to make you feel like a queen, with their stunning design and comfortable fit. Whether you're looking to add some excitement to your wardrobe or simply want to express your inner seductress, these boots are sure to do the trick.

The Femme-3020 boots feature a sexy thigh-high design with a lace-up front and a sturdy block heel. The platform measures 1 1/4 inches, giving you an extra boost of confidence and height. The heel is 8 inches tall, providing just the right amount of lift without sacrificing stability. The shaft of the boot measures approximately 26 inches from the arch, giving you plenty of leg coverage and support.

Made from high-quality synthetic materials, these boots are designed to be durable and long-lasting. The materials used are also incredibly comfortable, ensuring that you can wear these boots for hours on end without any discomfor

## 4. Run Inference on Endpoint

In [64]:
import scripts.prompt_utils as prompt_utils
from scripts.prompt_utils import format_prompt_instruct

In [65]:
import json

def invoke_sagemaker_endpoint(endpoint_name, payload, content_type='application/json'):
    """
    Call a deployed SageMaker endpoint.

    Parameters:
    - endpoint_name: Name of the deployed SageMaker endpoint.
    - payload: Input data for the model (can be a string or dict).
    - content_type: Content type of the input data (default is 'application/json').

    Returns:
    - Response from the model.
    """

    # If payload is a dictionary, convert it to a JSON string
    if isinstance(payload, dict):
        payload = json.dumps(payload)

    # Convert string payload to bytes
    # payload_bytes = payload.encode('utf-8')

    # Initialize a SageMaker runtime client
    sagemaker_runtime_client = boto3.client('sagemaker-runtime')

    # Invoke the SageMaker endpoint
    response = sagemaker_runtime_client.invoke_endpoint(
        EndpointName=endpoint_name,
        # Body=payload_bytes,
        Body=payload,
        ContentType = content_type, 
        CustomAttributes="accept_eula=true",
    )
    


    # Extract the result from the response and convert it to a dictionary
    result = json.loads(response['Body'].read().decode('utf-8'))

    return result

In [66]:
# define question and add to messages
# system_prompt = prompt_utils.system_prompt_tutor

# system_prompt = "You are a friendly and helpful accountinge tutor for MBA students. If the student does not ask a question about accounting please just act as a chatbot. If you feel like the provided context is not enough ask for more."
system_prompt = """Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation."""
user_instruction = "title: Devious Women's Femme 3020 34 inches Boots, brand: Devious, price: $68.76 - $145.50, feature: ['Synthetic', 'Shaft measures approximately 26 inches from arch', 'Heel Height: 8inches ', 'Shaft Height: 22 inches (size 7)', 'Fit: True to Size', 'Features of this item include: Sexy', 'Shipping Weight:\n                    \n6.5 pounds (View shipping rates and policies)'] [/INST] ['Shaft Height: Approx. 26\'\' Tall. Heel Height: Approx. 8\'\' Tall. The ultimate extension for those who live the lifestyle or those who just wish to spice it up again, Devious offers a fierce line of footwear to play with. The Femme-3020 boot has a unique block heel and 1 1/4 inches platform with a lace up thigh high design."

prompt = format_prompt_instruct(
    user_instruction = user_instruction,
    system_prompt = system_prompt,
)

body = {
    "inputs":  prompt,
  "parameters": {
    "do_sample": True,
    "top_p": 0.7,
    "temperature": 0.1,
    "top_k": 4,
    "max_new_tokens": 1000,
    "repetition_penalty": 1.03,
    "stop": ["</s>", "[/INST]"]
  }
}


# Example usage:
endpoint_name = 'huggingface-pytorch-tgi-inference-2024-01-19-19-18-47-920'
endpoint_llama = 'jumpstart-dft-meta-textgeneration-llama-2-13b-f'
response = invoke_sagemaker_endpoint(endpoint_name, body)
print(response[0]['generated_text'])


Discover the Devious Femme-3020 Boot! The ultimate extension for those who live the lifestyle or those who just wish to spice it up again, Devious offers a fierce line of footwear to play with. The Femme-3020 boot has a unique block heel and 1 1/4 inches platform with a lace up thigh high design. This boot is made of a durable stretch material that will fit most calf sizes. The Femme-3020 boot has a unique block heel and 1 1/4 inches platform with a lace up thigh high design. This boot is made of a durable stretch material that will fit most calf sizes. The Femme-3020 boot has a unique block heel and 1 1/4 inches platform with a lace up thigh high design. This boot is made of a durable stretch material that will fit most calf sizes. The Femme-3020 boot has a unique block heel and 1 1/4 inches platform with a lace up thigh high design. This boot is made of a durable stretch material that will fit most calf sizes.']


## 5. Test on foundation data questions

In [67]:
import pandas as pd

In [68]:
#data = pd.read_csv('s3://d3-generative-ai/foundation-testing/accounting/llama/llama-2-13b-chat/llama-2-13b-chat_accounting_20230829_finetuned.csv')
data = pd.read_csv('s3://d3-data-bucket/labs/trustworthy_ai/data/test_2.csv')
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   title                         17 non-null     object
 1   brand                         17 non-null     object
 2   feature                       17 non-null     object
 3   description                   17 non-null     object
 4   price                         17 non-null     object
 5   generated_description         17 non-null     object
 6   sample_generated_description  17 non-null     object
dtypes: object(7)
memory usage: 1.1+ KB


title               brand  \
0     X. L. Carbon Fiber Money Clip, made in the USA         Roar Carbon   
1               Buxton Heiress Pik-Me-Up Framed Case              Buxton   
2  Art Nouveau Sterling Silver Ornate Repousse He...     Silver Insanity   
3            Dream PJ's Blue - Large - Part #: 25BLG        Ethical/Spot   
4  Shimmer Anne Shine Clip On Costume/Halloween C...  Shimmer Anne Shine   

                                             feature  \
0  ['Real Carbon Fiber', 'Made in USA', '5 year w...   
1  ['Leather', 'Imported', 'synthetic lining', 'F...   
2  ['2&5/8" High and 3/4" Wide', 'Weight is Appro...   
3  ['Product Dimensions:\n                    \n8...   
4  ['Shimmer Anne Shine Clip On Costume/Halloween...   

                                         description    price  \
0  ['When you pull out your extra large carbon fi...  $14.99    
1  ['Authentic crunch leather with rich floral em...  $16.95    
2  ['It measures 2&5/8" tall x 3/4" wide, weighs ...  $44.66    
3  ['SOFT AND CUDDLY, SWEET DREAM PAJAMAS IN SOFT...  $15.99    
4  ['A fun addition to any costume party, play, o...   $6.99    

                               generated_description  \
0  Introducing the Schaefer Outfitters Ranger Ves...   
1  Sure, here's a vibrant and engaging product de...   
2  Introducing the Mickey Mantle "Mickey 7" Baseb...   
3  Introducing the Small World Toys Furree Faces ...   
4  Introducing the Klondike Sterling Nylon Trappe...   

                        sample_generated_description  
0  ["Introducing the X. L. Carbon Fiber Money Cli...  
1  ["Sure! Here's a vibrant and engaging product ...  
2  ['Introducing the Art Nouveau Sterling Silver ...  
3  ["Introducing the Dream PJ's Blue - Large, the...  
4  ["Introducing the Shimmer Anne Shine Clip On C...

In [69]:
import ast

def create_description_prompts(df):
    prompts = []
    for index, row in df.iterrows():
        # Check and convert the feature field
        feature = row['feature']
        if isinstance(feature, str):
            try:
                # Convert string representation of list to actual list
                feature_list = ast.literal_eval(feature)
            except (ValueError, SyntaxError):
                # If the string is not a list representation, make it a single-element list
                feature_list = [feature]
        elif isinstance(feature, list):
            feature_list = feature
        else:
            # If feature is neither a string nor a list, convert it to a string and make it a single-element list
            feature_list = [str(feature)]

        # Join the list elements into a string
        feature_list_str = '\n  - '.join(feature_list)
        
        prompt = (
            f"Title: {row['title']}\n"
            f"Brand: {row['brand']}\n"
            f"Price: ${row['price']}\n"
            f"Key Features: \n  - {feature_list_str}\n\n"
        )
        prompts.append(prompt)
    return prompts

In [70]:
#For generating only one description, use this block:

# Initialize the DataFrame with one column for finetuned_description
column_names = ['finetuned_description']
progress_df = pd.DataFrame(columns=column_names)
start_index = 0

# Generate prompts for each product in the part
prompts = create_description_prompts(data)

for index, example in enumerate(prompts, start=start_index):
    print('Starting generation for row: ' + str(index))
    try:
        user_instruction = example
        prompt = format_prompt_instruct(
            user_instruction=user_instruction,
            system_prompt=system_prompt
        )

        body = {
            "inputs":  prompt,
            "parameters": {
                "do_sample": True, "max_new_tokens": 512, "top_p": 0.9, "temperature": 1.0, "return_full_text": True
            }
        }

        response = predictor.predict(body)
        description = response[0]['generated_text']
    except Exception as e:
        print(f'Error: {e}')
        description = None  # Set description to None in case of error

    print('Description: ' + description)
    
    # Create a new row as a dictionary
    new_row = {'finetuned_description': description}

    # Add the new row to the DataFrame
    print('Completed iteration: ' + str(index))
    progress_df = progress_df.append(new_row, ignore_index=True)

Starting generation for row: 0
Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: X. L. Carbon Fiber Money Clip, made in the USA
Brand: Roar Carbon
Price: $$14.99 
Key Features: 
  - Real Carbon Fiber
  - Made in USA
  - 5 year warranty against delamination
  - Holds Cash and Cards
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
  - 

/tmp/ipykernel_12711/32764731.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: Buxton Heiress Pik-Me-Up Framed Case
Brand: Buxton
Price: $$16.95 
Key Features: 
  - Leather
  - Imported
  - synthetic lining
  - Flap closure
  - Authentic Crunch Leather
  - Rich Floral Embossed Logo
  - Goldtone Hardware
  - Large Snap Pocket
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused

/tmp/ipykernel_12711/32764731.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: Art Nouveau Sterling Silver Ornate Repousse Heart Bookmark - Book Lovers Gift
Brand: Silver Insanity
Price: $$44.66 
Key Features: 
  - 2&5/8" High and 3/4" Wide
  - Weight is Approx. 9 Grams
  - Marked 925
  - Vintage Inspired Style
  - Heart Bookmark
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and u

...

/tmp/ipykernel_12711/32764731.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: Stockman Vest
Brand: Schaefer Outfitters
Price: $$140.00 - $150.00
Key Features: 
  - Shipping Information:
                    
View shipping rates and policies

 [/INST] The Traditional Vest of ChoiceBy far the most popular stockman show vest on the market.This is the vest that most people think of when they think of the iconic Stockman Vest.Wear your Western pride in style.

/tmp/ipykernel_12711/32764731.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: CliC Adjustable Front Connect Universal Sport Goggle, Black Frame
Brand: CliC
Price: $$39.95 - $49.95
Key Features: 
  - Package Dimensions:
                    
10.2 x 5.9 x 2.6 inches
  - Shipping Weight:
                    
4 ounces (View shipping rates and policies)

 [/INST] FEATURES  With an ergonomic look and feel, this sleek, universal sports goggle offers 3 main ways

/tmp/ipykernel_12711/32764731.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


In [32]:
# Initialize the DataFrame with 20 columns for descriptions
column_names = ['description_' + str(i) for i in range(1, 21)]
progress_df = pd.DataFrame(columns=column_names)
start_index = 0

# Generate prompts for each product in the part
prompts = create_description_prompts(data)

for index, example in enumerate(prompts, start=start_index):
    print('Starting generation for row: ' + str(index))
    descriptions = {}  # Temporary dictionary to store descriptions
    for sample in range(1, 21):  # Loop to generate 20 samples per prompt
        try:
            user_instruction = example
            prompt = format_prompt_instruct(
                user_instruction=user_instruction,
                system_prompt=system_prompt
            )

            body = {
                "inputs":  prompt,
                "parameters": {
                    "do_sample": True, "max_new_tokens": 512, "top_p": 0.9, "temperature": 1.0, "return_full_text": True
                }
            }

            response = predictor.predict(body)
            description = response[0]['generated_text']
            descriptions['description_' + str(sample)] = description
        except Exception as e:
            print(f'Error: {e}')
            descriptions['description_' + str(sample)] = None
        
        print('Description: ' + description)
        
    # Create a new row with the 20 descriptions
    new_row = descriptions

    # Add the new row to the DataFrame
    print('Completed iteration: ' + str(index))
    progress_df = progress_df.append(new_row, ignore_index=True)

Starting generation for row: 0
Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: X. L. Carbon Fiber Money Clip, made in the USA
Brand: Roar Carbon
Price: $$14.99 
Key Features: 
  - Real Carbon Fiber
  - Made in USA
  - 5 year warranty against delamination
  - Holds Cash and Cards
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused condition.
  - 

/tmp/ipykernel_12711/4027126553.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: Buxton Heiress Pik-Me-Up Framed Case
Brand: Buxton
Price: $$16.95 
Key Features: 
  - Leather
  - Imported
  - synthetic lining
  - Flap closure
  - Authentic Crunch Leather
  - Rich Floral Embossed Logo
  - Goldtone Hardware
  - Large Snap Pocket
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and unused

/tmp/ipykernel_12711/4027126553.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: Art Nouveau Sterling Silver Ornate Repousse Heart Bookmark - Book Lovers Gift
Brand: Silver Insanity
Price: $$44.66 
Key Features: 
  - 2&5/8" High and 3/4" Wide
  - Weight is Approx. 9 Grams
  - Marked 925
  - Vintage Inspired Style
  - Heart Bookmark
  - You can return this item for any reason and get a full refund: no shipping charges. The item must be returned in new and u

...

/tmp/ipykernel_12711/4027126553.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: Stockman Vest
Brand: Schaefer Outfitters
Price: $$140.00 - $150.00
Key Features: 
  - Shipping Information:
                    
View shipping rates and policies

 [/INST] Maintaining the long-standing art of old time hunting gear, Schaefer Outfitters has devoted themselves to using only the best fabrics in order to reproduced one of the oldest and most popular pieces of hunti

/tmp/ipykernel_12711/4027126553.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


Description: <s>[INST] <<SYS>>
    Given a set of metadata about a product, craft a vibrant and engaging description. The description should transform the provided features into an alluring narrative, emphasizing the product's unique qualities. Focus on highlighting the practicality and quality of the product, along with any unique selling points. Your goal is to make the description irresistible to potential buyers. Remember to maintain a positive and persuasive tone throughout.Please provide only the product description in your response, without any additional commentary or explanation.
    <</SYS>>

    Title: CliC Adjustable Front Connect Universal Sport Goggle, Black Frame
Brand: CliC
Price: $$39.95 - $49.95
Key Features: 
  - Package Dimensions:
                    
10.2 x 5.9 x 2.6 inches
  - Shipping Weight:
                    
4 ounces (View shipping rates and policies)

 [/INST] Advanced technology and decades of experience go into the design of these goggles with quick-dryi

/tmp/ipykernel_12711/4027126553.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  progress_df = progress_df.append(new_row, ignore_index=True)


In [ ]:
'''results = []
for i, row in data[2:3].iterrows():
    print(f'{i}/{len(data)}')
    user_instruction = row['prompt']
    print(user_instruction)
    prompt = format_prompt_instruct(
        user_instruction = user_instruction,
        system_prompt = system_prompt
    )
    body = {
        "inputs":  prompt,
        "parameters": {
        "do_sample": True,
        "top_p": 0.6,
        "temperature": 0.4,
        "top_k": 4,
        "max_new_tokens": 1000,
        "repetition_penalty": 1.03,
        "stop": ["</s>"]
      }
    }

    try:
        chat = predictor.predict(body)
        results.append(chat[0]["generated_text"])
    except:
        print(f'Error: {i}')
        results.append(None)'''

    

In [71]:
progress_df

finetuned_description
0   <s>[INST] <<SYS>>\n    Given a set of metadata...
1   <s>[INST] <<SYS>>\n    Given a set of metadata...
2   <s>[INST] <<SYS>>\n    Given a set of metadata...
3   <s>[INST] <<SYS>>\n    Given a set of metadata...
4   <s>[INST] <<SYS>>\n    Given a set of metadata...
5   <s>[INST] <<SYS>>\n    Given a set of metadata...
6   <s>[INST] <<SYS>>\n    Given a set of metadata...
7   <s>[INST] <<SYS>>\n    Given a set of metadata...
8   <s>[INST] <<SYS>>\n    Given a set of metadata...
9   <s>[INST] <<SYS>>\n    Given a set of metadata...
10  <s>[INST] <<SYS>>\n    Given a set of metadata...
11  <s>[INST] <<SYS>>\n    Given a set of metadata...
12  <s>[INST] <<SYS>>\n    Given a set of metadata...
13  <s>[INST] <<SYS>>\n    Given a set of metadata...
14  <s>[INST] <<SYS>>\n    Given a set of metadata...
15  <s>[INST] <<SYS>>\n    Given a set of metadata...
16  <s>[INST] <<SYS>>\n    Given a set of metadata...

In [ ]:
'''results = []
for ind, user_instruction in enumerate(data['prompt']):
    print(f'{ind}/{len(data)}')
    print(user_instruction)
    prompt = format_prompt_instruct(
        user_instruction = user_instruction,
        system_prompt = system_prompt
    )
    body = {
        "inputs":  prompt,
      "parameters": {
        "do_sample": True,
        "top_p": 0.7,
        "temperature": 0.1,
        "top_k": 4,
        "max_new_tokens": 1000,
        "repetition_penalty": 1.03,
        "stop": ["</s>"]
      }
    }

    try:
        chat = predictor.predict(body)
        results.append(chat[0]["generated_text"])
        print(chat[0]["generated_text"])
    except:
        print(f'Error: {i}')
        results.append(None)'''

    

In [ ]:
'''response_dict = {'prompt': list(data['prompt']),
                 'finetuned_human_response':results}
final_dict = pd.DataFrame(response_dict)'''

In [ ]:
#final_dict.head()

In [72]:
progress_df

finetuned_description
0   <s>[INST] <<SYS>>\n    Given a set of metadata...
1   <s>[INST] <<SYS>>\n    Given a set of metadata...
2   <s>[INST] <<SYS>>\n    Given a set of metadata...
3   <s>[INST] <<SYS>>\n    Given a set of metadata...
4   <s>[INST] <<SYS>>\n    Given a set of metadata...
5   <s>[INST] <<SYS>>\n    Given a set of metadata...
6   <s>[INST] <<SYS>>\n    Given a set of metadata...
7   <s>[INST] <<SYS>>\n    Given a set of metadata...
8   <s>[INST] <<SYS>>\n    Given a set of metadata...
9   <s>[INST] <<SYS>>\n    Given a set of metadata...
10  <s>[INST] <<SYS>>\n    Given a set of metadata...
11  <s>[INST] <<SYS>>\n    Given a set of metadata...
12  <s>[INST] <<SYS>>\n    Given a set of metadata...
13  <s>[INST] <<SYS>>\n    Given a set of metadata...
14  <s>[INST] <<SYS>>\n    Given a set of metadata...
15  <s>[INST] <<SYS>>\n    Given a set of metadata...
16  <s>[INST] <<SYS>>\n    Given a set of metadata...

In [73]:
# Concatenate the DataFrames horizontally
final_dict = pd.concat([data, progress_df], axis=1)

In [74]:
final_dict.head()

title               brand  \
0     X. L. Carbon Fiber Money Clip, made in the USA         Roar Carbon   
1               Buxton Heiress Pik-Me-Up Framed Case              Buxton   
2  Art Nouveau Sterling Silver Ornate Repousse He...     Silver Insanity   
3            Dream PJ's Blue - Large - Part #: 25BLG        Ethical/Spot   
4  Shimmer Anne Shine Clip On Costume/Halloween C...  Shimmer Anne Shine   

                                             feature  \
0  ['Real Carbon Fiber', 'Made in USA', '5 year w...   
1  ['Leather', 'Imported', 'synthetic lining', 'F...   
2  ['2&5/8" High and 3/4" Wide', 'Weight is Appro...   
3  ['Product Dimensions:\n                    \n8...   
4  ['Shimmer Anne Shine Clip On Costume/Halloween...   

                                         description    price  \
0  ['When you pull out your extra large carbon fi...  $14.99    
1  ['Authentic crunch leather with rich floral em...  $16.95    
2  ['It measures 2&5/8" tall x 3/4" wide, weighs ...  $44.66    
3  ['SOFT AND CUDDLY, SWEET DREAM PAJAMAS IN SOFT...  $15.99    
4  ['A fun addition to any costume party, play, o...   $6.99    

                               generated_description  \
0  Introducing the Schaefer Outfitters Ranger Ves...   
1  Sure, here's a vibrant and engaging product de...   
2  Introducing the Mickey Mantle "Mickey 7" Baseb...   
3  Introducing the Small World Toys Furree Faces ...   
4  Introducing the Klondike Sterling Nylon Trappe...   

                        sample_generated_description  \
0  ["Introducing the X. L. Carbon Fiber Money Cli...   
1  ["Sure! Here's a vibrant and engaging product ...   
2  ['Introducing the Art Nouveau Sterling Silver ...   
3  ["Introducing the Dream PJ's Blue - Large, the...   
4  ["Introducing the Shimmer Anne Shine Clip On C...   

                                       description_1  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                       description_2  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                       description_3  ...  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   

                                      description_12  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                      description_13  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                      description_14  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                      description_15  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n  

In [37]:
final_dict.to_csv("s3://d3-data-bucket/labs/trustworthy_ai/data/19thJan_finetuned_generated_descriptions.csv",index=False)

In [ ]:
#chat[0]["generated_text"]

In [ ]:
#data['response_finetuned'] = results
#data.to_csv('s3://d3-data-bucket/labs/trustworthy_ai/data/llama-2-13b-chat_finetuned.csv')

## 6. Delete endpoint

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()

In [ ]:
predictor

In [41]:
pwd

'/root/DemoNotebooks/hallucinations/Llama-customize/training'

In [51]:
df = pd.read_csv('data/19thJan_finetuned_generated_descriptions.csv')

In [47]:
import re
# Using a regular expression to match the end of the key features list and extract the product description
def extract_description_after_features(text):
    # Check if the text is a string
    if not isinstance(text, str):
        return text  # Or return str(text) to convert non-strings to strings

    # Match everything after the key features list, assuming it ends with the closing bracket of the list
    match = re.search(r'[/INST] ', text, re.DOTALL)
    if match:
        # Extract the text after the key features list
        return text[match.end():].strip()
    else:
        # If no match is found, return the original text indicating there was no key features list
        return text

# Apply the extraction function to every description in the dataframe
for col in df.columns:
    df[col] = df[col].apply(extract_description_after_features)

In [44]:
# Define a function to remove '[/INST]' from each cell
def remove_inst_tag(text):
    # Check if the text is a string
    if isinstance(text, str):
        # Remove the '[/INST]' tag
        return text.replace('[/INST] ', '')
    else:
        return text

# Apply the function to every cell in the dataframe
for col in df.columns:
    df[col] = df[col].apply(remove_inst_tag)

# Display the first few rows of the updated dataframe to verify the changes
df.head()

title               brand  \
0     X. L. Carbon Fiber Money Clip, made in the USA         Roar Carbon   
1               Buxton Heiress Pik-Me-Up Framed Case              Buxton   
2  Art Nouveau Sterling Silver Ornate Repousse He...     Silver Insanity   
3            Dream PJ's Blue - Large - Part #: 25BLG        Ethical/Spot   
4  Shimmer Anne Shine Clip On Costume/Halloween C...  Shimmer Anne Shine   

                                             feature  \
0  ['Real Carbon Fiber', 'Made in USA', '5 year w...   
1  ['Leather', 'Imported', 'synthetic lining', 'F...   
2  ['2&5/8" High and 3/4" Wide', 'Weight is Appro...   
3  ['Product Dimensions:\n                    \n8...   
4  ['Shimmer Anne Shine Clip On Costume/Halloween...   

                                         description    price  \
0  ['When you pull out your extra large carbon fi...  $14.99    
1  ['Authentic crunch leather with rich floral em...  $16.95    
2  ['It measures 2&5/8" tall x 3/4" wide, weighs ...  $44.66    
3  AND CUDDLY, SWEET DREAM PAJAMAS IN SOFT PASTEL...  $15.99    
4  ['A fun addition to any costume party, play, o...   $6.99    

                               generated_description  \
0  Introducing the Schaefer Outfitters Ranger Ves...   
1  Sure, here's a vibrant and engaging product de...   
2  Introducing the Mickey Mantle "Mickey 7" Baseb...   
3  Introducing the Small World Toys Furree Faces ...   
4  Introducing the Klondike Sterling Nylon Trappe...   

                        sample_generated_description  \
0  get compliments every time I use it', "Introdu...   
1  ["Sure! Here's a vibrant and engaging product ...   
2  ['Introducing the Art Nouveau Sterling Silver ...   
3  ["Introducing the Dream PJ's Blue - Large, the...   
4  ["Introducing the Shimmer Anne Shine Clip On C...   

                                       description_1  \
0  AMG. Although it is lightweight, it is up to f...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  chose the soft organic cotton in a variety of ...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                       description_2  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                       description_3  ...  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
3                                       at checkout.  ...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   

                                      description_11  \
0  - Carbon Fiber. - Dimensions: 2 x 2 inches. - ...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  EASY!Our classic Dream PJs have people all aro...   
4  AND SAFE FOR YOUR KIDS!! The wires are very fl...   

                                      description_12  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  JAMMIES IN AID OF CHILDREN'S RIGHTS - Slavery ...   
4  GUARANTEE: Shimmer Anne Shine is proud to stan...   

                                      description_13  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                      description_14  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n  

In [48]:
# Display the first few rows of the updated dataframe to verify the changes
df.head()

title               brand  \
0     X. L. Carbon Fiber Money Clip, made in the USA         Roar Carbon   
1               Buxton Heiress Pik-Me-Up Framed Case              Buxton   
2  Art Nouveau Sterling Silver Ornate Repousse He...     Silver Insanity   
3            Dream PJ's Blue - Large - Part #: 25BLG        Ethical/Spot   
4  Shimmer Anne Shine Clip On Costume/Halloween C...  Shimmer Anne Shine   

                                             feature  \
0  ['Real Carbon Fiber', 'Made in USA', '5 year w...   
1  ['Leather', 'Imported', 'synthetic lining', 'F...   
2  ['2&5/8" High and 3/4" Wide', 'Weight is Appro...   
3  ['Product Dimensions:\n                    \n8...   
4  ['Shimmer Anne Shine Clip On Costume/Halloween...   

                                         description    price  \
0  ['When you pull out your extra large carbon fi...  $14.99    
1  ['Authentic crunch leather with rich floral em...  $16.95    
2  ['It measures 2&5/8" tall x 3/4" wide, weighs ...  $44.66    
3  AND CUDDLY, SWEET DREAM PAJAMAS IN SOFT PASTEL...  $15.99    
4  ['A fun addition to any costume party, play, o...   $6.99    

                               generated_description  \
0  Introducing the Schaefer Outfitters Ranger Ves...   
1  Sure, here's a vibrant and engaging product de...   
2  Introducing the Mickey Mantle "Mickey 7" Baseb...   
3  Introducing the Small World Toys Furree Faces ...   
4  Introducing the Klondike Sterling Nylon Trappe...   

                        sample_generated_description  \
0  get compliments every time I use it', "Introdu...   
1  ["Sure! Here's a vibrant and engaging product ...   
2  ['Introducing the Art Nouveau Sterling Silver ...   
3  ["Introducing the Dream PJ's Blue - Large, the...   
4  ["Introducing the Shimmer Anne Shine Clip On C...   

                                       description_1  \
0  AMG. Although it is lightweight, it is up to f...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  chose the soft organic cotton in a variety of ...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                       description_2  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                       description_3  ...  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   
3                                       at checkout.  ...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...  ...   

                                      description_11  \
0  - Carbon Fiber. - Dimensions: 2 x 2 inches. - ...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  EASY!Our classic Dream PJs have people all aro...   
4  AND SAFE FOR YOUR KIDS!! The wires are very fl...   

                                      description_12  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  JAMMIES IN AID OF CHILDREN'S RIGHTS - Slavery ...   
4  GUARANTEE: Shimmer Anne Shine is proud to stan...   

                                      description_13  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n    Given a set of metadata...   
2  <s>[INST] <<SYS>>\n    Given a set of metadata...   
3  <s>[INST] <<SYS>>\n    Given a set of metadata...   
4  <s>[INST] <<SYS>>\n    Given a set of metadata...   

                                      description_14  \
0  <s>[INST] <<SYS>>\n    Given a set of metadata...   
1  <s>[INST] <<SYS>>\n  

In [55]:
# Save the updated dataframe to a new CSV file
df.to_csv('/root/DemoNotebooks/hallucinations/Llama-customize/training/data/19thJan_finetuned_data.csv', index=False)

In [77]:
import pandas as pd

df = pd.read_csv('data/19thJan_finetuned_generated_descriptions.csv')

# Define a function to extract text after [/INST]
def extract_text_after_inst(text):
    if pd.isna(text):
        return text  # Return NaN as is
    parts = text.split('[/INST]')
    return parts[-1].strip() if len(parts) > 1 else text

# Apply this function to each relevant column
for column in df.columns:
    if column.startswith('description_'):
        df[column] = df[column].apply(extract_text_after_inst)


In [78]:
final_dict.head()

title               brand  \
0     X. L. Carbon Fiber Money Clip, made in the USA         Roar Carbon   
1               Buxton Heiress Pik-Me-Up Framed Case              Buxton   
2  Art Nouveau Sterling Silver Ornate Repousse He...     Silver Insanity   
3            Dream PJ's Blue - Large - Part #: 25BLG        Ethical/Spot   
4  Shimmer Anne Shine Clip On Costume/Halloween C...  Shimmer Anne Shine   

                                             feature  \
0  ['Real Carbon Fiber', 'Made in USA', '5 year w...   
1  ['Leather', 'Imported', 'synthetic lining', 'F...   
2  ['2&5/8" High and 3/4" Wide', 'Weight is Appro...   
3  ['Product Dimensions:\n                    \n8...   
4  ['Shimmer Anne Shine Clip On Costume/Halloween...   

                                         description    price  \
0  ['When you pull out your extra large carbon fi...  $14.99    
1  ['Authentic crunch leather with rich floral em...  $16.95    
2  ['It measures 2&5/8" tall x 3/4" wide, weighs ...  $44.66    
3  ['SOFT AND CUDDLY, SWEET DREAM PAJAMAS IN SOFT...  $15.99    
4  ['A fun addition to any costume party, play, o...   $6.99    

                               generated_description  \
0  Introducing the Schaefer Outfitters Ranger Ves...   
1  Sure, here's a vibrant and engaging product de...   
2  Introducing the Mickey Mantle "Mickey 7" Baseb...   
3  Introducing the Small World Toys Furree Faces ...   
4  Introducing the Klondike Sterling Nylon Trappe...   

                        sample_generated_description  \
0  ["Introducing the X. L. Carbon Fiber Money Cli...   
1  ["Sure! Here's a vibrant and engaging product ...   
2  ['Introducing the Art Nouveau Sterling Silver ...   
3  ["Introducing the Dream PJ's Blue - Large, the...   
4  ["Introducing the Shimmer Anne Shine Clip On C...   

                                       description_1  \
0  Made in the USA with real Carbon Fiber. This m...   
1  To accompany your Built-to-Order Buxton bag, o...   
2  Your book group loved "The Great Gatsby"? Do y...   
3  Dream PJ's is a wardrobe staple that offers th...   
4  Snap these costume cat ears onto your head for...   

                                       description_2  \
0  No more rummaging through your pockets and coa...   
1  This gorgeous case from Buxton's Signature Fas...   
2  Crafted in high polished sterling silver with ...   
3  Pure cotton COTTON, no blends, no synthetics. ...   
4  Shimmer Anne Shine Costume/Halloween Clip On C...   

                                       description_3  ...  \
0  X.L. Carbon Fiber Money Clip, Made in the USA ...  ...   
1  Buy any 3 items from Buxton.com and get 1 on u...  ...   
2  Intricately designed in a Renaissance style th...  ...   
3  P.J's Dream can help your child to dream big. ...  ...   
4  Shimmer Anne Shine are proud to present our wo...  ...   

                                      description_12  \
0  X.L. Carbon Fiber Money Clip. Made in the USA....   
1  The Buxton Heiress Pik-Me-Up Framed Case, avai...   
2  A charming repousse vintage Art Nouveau sterli...   
3  LUXURY COTTON JAMMIES IN AID OF CHILDREN'S RIG...   
4  Have a blast for the Halloween season or other...   

                                      description_13  \
0  There's a certain something that can be said f...   
1  The Buxton Heiress Pik-Me-Up Framed Case is a ...   
2  You can't always judge a book by its cover, bu...   
3  Our top selling and most comfortable shirt. Si...   
4  Shimmer Anne Shine Clip On Costume/Halloween C...   

                                      description_14  \
0  Your new favorite money clip is made from a ne...   
1  Snap it up!  An adjustable Velcro clasp in the...   
2  Open up a book and uncover a work of art with ...   
3  Snuggly is the best word to describe Dream PJ'...   
4  Comfortable, seamless hair accessories \nClip ...   

                                      description_15  \
0  Made entirely of real carbon fiber and finishe...   
1  Buy a Heart Shaped Eg

In [80]:
# Save the modified DataFrame to a new CSV file
output_file_path = 'data/finetuned_19thJan_final.csv'
final_dict.to_csv(output_file_path, index=False)

In [79]:
pwd

'/root/DemoNotebooks/hallucinations/Llama-customize/training'